In [ ]:
import numpy as np
import pandas as pd
import tweepy
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
consumerKey = os.getenv("consumerKey")
consumerSecret = os.getenv("consumerSecret")
accessToken = os.getenv("accessToken")
accessTokenSecret = os.getenv("accessTokenSecret")
location = os.getenv("location")

In [ ]:
def pulling(location, noOfTweet):
    auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
    auth.set_access_token(accessToken, accessTokenSecret)
    api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

    keyword = f'vaccine {location}'
    tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
    print(f'getting data...')
    tweet_list = [[tweet.user.screen_name, tweet.user.followers_count, tweet.user.friends_count, 
                    tweet.retweet_count, tweet.favorite_count, tweet.text] for tweet in tweets]
    tweet_text = pd.DataFrame(data=tweet_list, columns=['user', "followers_count", "friends_count", 
                                                        "retweet_count", "favorite_count", "text"])
    tweet_text.to_csv(f'./dataset/tweet-df-{location}.csv', index=False)

    return tweet_text

In [ ]:
noOfTweet = 1000
df = pulling(location, noOfTweet)

In [ ]:
df = pd.read_csv(f'./dataset/tweet-df-{location}.csv', sep=',', lineterminator='\n')
print(df)

In [ ]:
retweeted = []
user = []
user_followers = []
text = []
for index, row in df.text.iteritems():
    if "RT @" in row:
        username = row.split("RT @",1)[1].split(":",1)[-2]
        if " " in username:
            username = username.split(" ",1)[-2]
        retweeted.append(username)
        user.append(df.user[index])
        user_followers.append(df.followers_count[index])
        text.append(df.text[index])
edge = pd.DataFrame(zip(user, user_followers, retweeted, text), columns =['user', 'user_followers', 'retweeted_user', 'text'])
edge = edge.drop(edge[edge.user == edge.retweeted_user].index)
edge['retweeted_user_followers'] = ""
edge['sentiment'] = ""

In [ ]:
retweeted_user_list = list(edge.retweeted_user)
retweeted_user_list = list(dict.fromkeys(retweeted_user_list))
# print(retweeted_user_list)

In [ ]:
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

retweeted_user_followers_dict = {}

for screen_name in retweeted_user_list:
    # fetching the user
    print(f'fetching {screen_name}...')
    user = api.get_user(screen_name, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
    # fetching the followers_count
    followers_count = user.followers_count

    retweeted_user_followers_dict[screen_name] = followers_count

In [ ]:
retweeted_user_followers_list = [retweeted_user_followers_dict[i] for i in list(edge.retweeted_user)]
edge['retweeted_user_followers'] = retweeted_user_followers_list
edge.to_csv(f'./dataset/edge-{location}.csv', index=False)